In [ ]:
import numpy as np
import os
import csv
import ftplib as ftp
import sys
import codecs
import time
import pandas as pd
import enum
import pickle as pk
from neo4jrestclient.client import GraphDatabase
from neo4jrestclient import client
from flask import Flask
from tabulate import tabulate as tb
app = Flask(__name__)

In [ ]:
db = GraphDatabase("<localhost>",username="",password="")

In [ ]:
q_baddress = """match (p:substreet)
where p.substreetId='%s'
return p.address1, p.address2, p.city,p.state,p.zipcode"""

q_business_info = """match  (badd:substreet) <- [:business_address] - (n:npi) - [:business_phone] -> (p:phone)
where n.ind_or_org = "2" and badd.substreetId = '%s'
return distinct(n.org_legal_name), badd.address1, badd.address2, badd.city, badd.state, badd.zipcode, collect (distinct p.name) as bus_phones
"""
q_num_prac_addr = """match  (badd:substreet) <- [:business_address] - (n:npi) - [:practice_address] -> (prad:substreet)
where n.ind_or_org = "2" and badd.substreetId = '%s'
//with badd, n, padd
//match (n) - [:npi_taxonomy] - (t:taxonomy)
return distinct(n.org_legal_name), count(distinct(prad)) as num_practice_addresses
"""

q_loc_prac="""match  (badd:substreet) <- [:business_address] - (n:npi) - [:practice_address] -> (padd:substreet) <- [:practice_address] - (pn:npi)
where  badd.substreetId = '%s' and n.ind_or_org = '2' and pn.ind_or_org = '2'
with collect({loc:badd.substreetId, name: n.org_legal_name, other:n.other_org_name}) as no, collect({loc:padd.substreetId, name:pn.org_legal_name, other:pn.other_org_name}) as np
unwind (no + np) as nall
with nall.name as name, nall.loc as loc,  collect(distinct nall.other) as other
return distinct loc, name,  other
order by loc
"""

q_top_tax="""match  (badd:substreet) <- [:practice_address] - (n:npi) - [:npi_taxonomy] -> (t:taxonomy)
where  badd.substreetId = '%s' 
with collect({name:t.classification, npiname:n.name}) as s1
match  (badd:substreet) <- [:business_address] - (n:npi) - [:practice_address] -> (padd:substreet)
where  badd.substreetId = '%s'  and n.ind_or_org = '2'
with padd, s1
match (t:taxonomy)<- [:npi_taxonomy]-  (n:npi) - [:practice_address] -> (padd)
with s1, collect({name:t.classification, npiname:n.name}) as s2
unwind (s1 + s2) as rows
return distinct(rows.name), count(distinct rows.npiname) as ct
order by ct desc
limit 30
"""

In [ ]:
with codecs.open('Exploring - Sheet1.csv','rU','utf-8') as csvfile:
    read_leadf = csv.DictReader(csvfile)
    cnt=0

    company_details=""

    for r in read_leadf:
        ###business address
        baddress=db.query(q_baddress%(r["loc"]))
        print("\n\n\t\t\t\t\t\033[1mCompany Profile %s\033[0m\n\n"%str(cnt+1))
        company_details+="\033[1m\033[4mBusiness/Billing Address\033[0m \n\n"
        for i in baddress[0]:
            company_details+=i+"\n"
        #####################
        
        ###Business name and phone
        company_details+="\n\033[1m\033[4mPhone Numbers\033[0m \n"
        company_details+="\n\033[1mBusiness_Org_Legal_Name\033[0m\t"+"\t||\t"+"\033[1mPhone_Numbers\033[0m\t\n"
        bss_phone = db.query(q_business_info%r["loc"])
        
        for i in list(bss_phone):
            company_details+=str(i[0])+"\t||\t"
            for j in (i[-1]):
                company_details+=str(j)+", "
            company_details=company_details[:-2]
            company_details+="\t\n\n"
        ##########################
        
        ###Number of practice locations for the Business address
        company_details+="\n\033[1m\033[4mNumber of Practice Locations for the Business\033[0m \n\n"
        num_prac = db.query(q_num_prac_addr%r["loc"])
        company_details+=tb(list(num_prac),headers=["Org Legal Name","Number of Practice locations"],tablefmt='orgtbl')
        ########################################################
        
        ###Practice location names
        company_details+="\n\n\033[1m\033[4mNames of Practice Locations\033[0m\n"
        company_details+="\n\033[1mOrg_Legal_Name\033[0m\t"+"\t||\t"+"\033[1mNames_of_Practice_Locations_for this_business\033[0m\t\n"
        loc_prac = (db.query(q_loc_prac%r["loc"]))
        
        for i in list(loc_prac):
            company_details+=str(i[1])+"\t||\t"
            for j in (i[-1]):
                company_details+=str(j)+", "
            company_details=company_details[:-2]
            company_details+="\t\n\n"
        ##########################
        
        ###Most Relevant Taxonomies
        company_details+="\n\033[1m\033[4mMost Relevant Taxonomies in the Business Location\033[0m \n\n"
        top_tax=db.query(q_top_tax%(r["loc"],r["loc"]))
        company_details+=tb(list(top_tax),headers=["Taxonomy Type","Number"],tablefmt='orgtbl')
        ###########################
        
        print(company_details,end="\r")
        print("\n\n\n")
        company_details=""
        #if(cnt==5):break
        cnt+=1
        
        if(cnt==57):break
                
